In [71]:
import requests
import pandas as pd
from tqdm import tqdm
import folium

In [47]:
url_inventory_txt = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-inventory.txt"
url_stations_txt = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt"
url_countries_txt = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-countries.txt"
url_states_txt = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-states.txt"
url_state_raw = "https://raw.githubusercontent.com/georgique/world-geojson/develop/states/usa/new_hampshire.json"

In [51]:
def parse_stations_txt(url):
    r = requests.get(url)
    lines = r.text.split("\n")
    data = []
    for line in lines:
        if line:
            data.append({
                "ID": line[0:11].strip(),
                "LATITUDE": float(line[12:20].strip()),
                "LONGITUDE": float(line[21:30].strip()),
                "ELEVATION": float(line[31:37].strip()),
                "STATE": line[38:40].strip(),
                "NAME": line[38:68].strip(),
            })
    return pd.DataFrame(data)

def parse_inventory_txt(url):
    r = requests.get(url)
    lines = r.text.split("\n")
    data = []
    for line in lines:
        if line:
            data.append({
                "ID": line[0:11].strip(),
                "LATITUDE": float(line[12:20].strip()),
                "LONGITUDE": float(line[21:30].strip()),
                "ELEMENT": line[31:35].strip(),
                "FIRSTYEAR": int(line[36:40].strip()),
                "LASTYEAR": int(line[41:45].strip())
            })
    return pd.DataFrame(data)

def parse_countries_txt(url):
    r = requests.get(url)
    lines = r.text.split("\n")
    data = []
    for line in lines:
        if line:
            data.append({
                "CODE": line[0:2].strip(),
                "NAME": line[3:64].strip()
            })
    return pd.DataFrame(data)

def parse_states_txt(url):
    r = requests.get(url)
    lines = r.text.split("\n")
    data = []
    for line in lines:
        if line:
            data.append({
                "CODE": line[0:2].strip(),
                "NAME": line[3:50].strip()
            })
    return pd.DataFrame(data)

def parse_data_dly(line):
    data = []
    for i in range(21, 269, 8):
        value = int(line[i:i+5])/10
        mflag = line[i+5]
        qflag = line[i+6]
        sflag = line[i+7]
        data.extend([value, mflag, qflag, sflag])
    return {
        "ID": line[0:11].strip(),
        "YEAR": int(line[11:15]),
        "Month": int(line[15:17]),
        "ELEMENT": line[17:21].strip(),
        "DATA": data
    }

def read_data_from_url(url):
    data = []
    response = requests.get(url)
    if response.status_code == 200:
        for line in response.text.splitlines():
            data.append(parse_data_dly(line))
    else:
        print(f"Failed to retrieve data for {url}. Status code: {response.status_code}")
    return data

def fetch_and_save_to_dataframe(station_ids):
    all_data = []
    for station_id in tqdm(station_ids, desc="Fetching Data", unit="station", ncols=100):
        url = f"https://www.ncei.noaa.gov/pub/data/ghcn/daily/all/{station_id}.dly"
        data = read_data_from_url(url)
        all_data.extend(data)
    headers = ["ID", "YEAR", "Month", "ELEMENT"]
    for i in range(1, 32):
        headers.extend([f"VALUE{i}", f"MFLAG{i}", f"QFLAG{i}", f"SFLAG{i}"])
    df_data = []
    for entry in all_data:
        row = [entry["ID"], entry["YEAR"], entry["Month"], entry["ELEMENT"]]
        row.extend(entry["DATA"])
        df_data.append(row)
    return pd.DataFrame(df_data, columns=headers)


In [40]:
inventory = parse_inventory_txt(url_inventory_txt)
stations= parse_stations_txt(url_stations_txt)
countries = parse_countries_txt(url_countries_txt)
states = parse_states_txt(url_states_txt)

In [49]:
# For country
#s_country_list = stations[stations['ID'].str.startswith('US')]['ID'].tolist()
# For state
s_state_list=stations[stations['STATE']=='NH']['ID'].tolist()
s_live_list=inventory[(inventory['ID'].isin(s_state_list)) & (inventory['LASTYEAR']>=2024)]['ID'].unique().tolist()
result = inventory.loc[inventory['ID'].isin(s_live_list), ['ID', 'ELEMENT', 'FIRSTYEAR', 'LASTYEAR']]
result['YEAR_DIFF'] = result['LASTYEAR'] - result['FIRSTYEAR']
result_filtered = result[result['ELEMENT'].isin([ 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'PRCP'])]
Req_station_list = pd.Series(result_filtered[(result_filtered['YEAR_DIFF'] >= 70) & (result_filtered['LASTYEAR'] > 2024)]['ID'].tolist()).unique()

In [ ]:
data= fetch_and_save_to_dataframe(Req_station_list)

In [88]:
flag_columns = [col for col in data.columns if 'FLAG' in col]
data= data.drop(columns=flag_columns)

In [61]:
def transform_weather_data(df):
    transformed_data = {}
    for index, row in df.iterrows():
        for day in range(1, 32):
            date = pd.to_datetime(f"{row['YEAR']}-{row['Month']:02d}-{day:02d}", errors='coerce')
            if pd.notna(date):
                if date not in transformed_data:
                    transformed_data[date] = {"ID": row['ID']}
                element = row['ELEMENT']
                value = row.get(f'VALUE{day}', None)
                if pd.notna(value) and value != -999.9:
                    transformed_data[date][element] = value
    return pd.DataFrame.from_dict(transformed_data, orient='index').reset_index().rename(columns={'index': 'DATE'})

In [ ]:
state_stations = stations[stations['STATE']=='NH']
state_lat = state_stations['LATITUDE'].mean()
state_lon = state_stations['LONGITUDE'].mean()
state_map = folium.Map(location=[state_lat, state_lon], 
                       zoom_start=7.5,
                       min_zoom=6,
                       max_zoom=10)

stations_to_plot = stations[stations['ID'].isin(Req_station_list)]
for _, row in stations_to_plot.iterrows():
    folium.Marker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        popup=row['NAME'],
        tooltip=row['NAME']
    ).add_to(state_map)
try:
    geojson_data = requests.get(url_state_raw).json() 
    folium.GeoJson(geojson_data, name="State Boundary").add_to(state_map)
except Exception as e:
    print(f"Error loading GeoJSON: {e}")
    
state_map

In [ ]:
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

# Apply the function to the 'month' column
data['season'] = data['Month'].apply(get_season)

# Now you can split the data by season
winter_data = data[data['season'] == 'Winter']
spring_data = data[data['season'] == 'Spring']
summer_data = data[data['season'] == 'Summer']
fall_data = data[data['season'] == 'Fall']

In [ ]:
# Example of accessing the split data
winter_data[(winter_data['YEAR'] == 2024 )& (winter_data['Month']==2)].head(20)

In [ ]:
# Group by 'station' and 'element', then count occurrences of each element per station
element_counts_by_station = data.groupby(['ID', 'ELEMENT']).size().unstack(fill_value=0)

# Print the result
element_counts_by_station[['SNOW', 'SNWD', 'WESD', 'PRCP', 'TMAX', 'TMIN', 'ADPT']].sum(axis=1).sort_values(ascending=False).head(10)

# Individual counts for each element
element_counts_by_station[['SNOW', 'SNWD', 'WESD', 'PRCP', 'TMAX', 'TMIN', 'ADPT']]




In [ ]:
# Remove duplicates
data_unique = data.drop_duplicates(subset=['ID', 'YEAR', 'Month'])

# Group by 'ID' and count the number of rows for each station
rows_per_station = data_unique.groupby('ID').size()

print(rows_per_station)